In [2]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np

In [3]:
client = MongoClient("mongodb://localhost:27017/")

In [4]:
db = client.companies

In [5]:
colletion_companies = db.limpiado

In [6]:
query = colletion_companies.find()

In [7]:
data2 = pd.DataFrame(query)

In [8]:
data2.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,5d848bb6eacafe60c6657beb,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ..."
1,5d848bb6eacafe60c6657bec,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,..."
2,5d848bb6eacafe60c6657bed,Zango,games_video,225,1999,,$40M,"[{'description': None, 'address1': '3600 136th...",-122.155668,47.579208,"{'type': 'Point', 'coordinates': [-122.155668,..."
3,5d848bb6eacafe60c6657bee,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,..."
4,5d848bb6eacafe60c6657bef,Crunchyroll,games_video,50,2008,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2...",-122.393229,37.781265,"{'type': 'Point', 'coordinates': [-122.393229,..."


In [9]:
data2['position'].head()


0    {'type': 'Point', 'coordinates': [-73.995625, ...
1    {'type': 'Point', 'coordinates': [-122.206893,...
2    {'type': 'Point', 'coordinates': [-122.155668,...
3    {'type': 'Point', 'coordinates': [-118.254558,...
4    {'type': 'Point', 'coordinates': [-122.393229,...
Name: position, dtype: object

In [21]:
data2['position'][4]

{'type': 'Point', 'coordinates': [-122.393229, 37.781265]}

In [10]:
data2["cities"] = [e[0]["city"] for e in data2["offices"]]
data2['cities'].value_counts().head()

San Francisco    10
New York          9
Los Angeles       5
Miami             4
Paris             3
Name: cities, dtype: int64

In [11]:
data2.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position,cities
0,5d848bb6eacafe60c6657beb,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ...",New York
1,5d848bb6eacafe60c6657bec,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,...",Redwood City
2,5d848bb6eacafe60c6657bed,Zango,games_video,225,1999,,$40M,"[{'description': None, 'address1': '3600 136th...",-122.155668,47.579208,"{'type': 'Point', 'coordinates': [-122.155668,...",Bellevue
3,5d848bb6eacafe60c6657bee,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,...",Los Angeles
4,5d848bb6eacafe60c6657bef,Crunchyroll,games_video,50,2008,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2...",-122.393229,37.781265,"{'type': 'Point', 'coordinates': [-122.393229,...",San Francisco


In [13]:
#import geonear
def geonear(geopoint, maxdistance=1000):
   return db.limpiado.find({
       "position":{
           "$near":{
               "$geometry":geopoint,
               "$maxDistance":maxdistance
           }}})

In [14]:
San_Francisco = {'type': 'Point', 'coordinates': [-122.393229, 37.781265]}

san = pd.DataFrame(geonear(San_Francisco, 5000))

In [15]:
san.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,5d848bb6eacafe60c6657bef,Crunchyroll,games_video,50,2008,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2...",-122.393229,37.781265,"{'type': 'Point', 'coordinates': [-122.393229,..."
1,5d848bb6eacafe60c6657c0e,iWin,games_video,103,2001,,$0,"[{'description': '', 'address1': '45 Fremont S...",-122.391061,37.781689,"{'type': 'Point', 'coordinates': [-122.391061,..."
2,5d848bb6eacafe60c6657bf8,Curse,games_video,58,2006,Online games media company,$12M,"[{'description': 'San Francisco', 'address1': ...",-122.399972,37.787092,"{'type': 'Point', 'coordinates': [-122.399972,..."
3,5d848bb6eacafe60c6657bf9,hi5,games_video,100,2003,,$52M,"[{'description': '', 'address1': '55 Second St...",-122.400558,37.788668,"{'type': 'Point', 'coordinates': [-122.400558,..."
4,5d848bb6eacafe60c6657c12,ngmoco,games_video,150,2008,Games for the iPhone,$44.8M,"[{'description': 'Secret Factory', 'address1':...",-122.401581,37.770510,"{'type': 'Point', 'coordinates': [-122.4015807..."


In [36]:
for g in san.position:
    print(g)
    san2 = list(geonear(g, 2000))

{'type': 'Point', 'coordinates': [-122.393229, 37.781265]}
{'type': 'Point', 'coordinates': [-122.391061, 37.781689]}
{'type': 'Point', 'coordinates': [-122.399972, 37.787092]}
{'type': 'Point', 'coordinates': [-122.400558, 37.788668]}
{'type': 'Point', 'coordinates': [-122.4015807, 37.7705096]}
{'type': 'Point', 'coordinates': [-122.409173, 37.788482]}
{'type': 'Point', 'coordinates': [-122.404234, 37.765158]}


In [37]:
san2

[{'_id': ObjectId('5d848bb6eacafe60c6657c01'),
  'name': 'Zynga',
  'category_code': 'games_video',
  'number_of_employees': 115,
  'founded_year': 2007,
  'description': 'Network of gaming applications',
  'total_money_raised': '$860M',
  'offices': [{'description': 'SF HQ',
    'address1': '365 Vermont St.',
    'address2': '',
    'zip_code': '94103',
    'city': 'San Francisco',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.765158,
    'longitude': -122.404234},
   {'description': '',
    'address1': '',
    'address2': '',
    'zip_code': '',
    'city': '',
    'state_code': None,
    'country_code': 'IND',
    'latitude': None,
    'longitude': None},
   {'description': '',
    'address1': '',
    'address2': '',
    'zip_code': '',
    'city': 'Baltimore',
    'state_code': 'MD',
    'country_code': 'USA',
    'latitude': None,
    'longitude': None},
   {'description': '',
    'address1': '',
    'address2': '',
    'zip_code': '',
    'city': 'Los Ange

In [38]:
san['number_of_employees'] >= 100

0    False
1     True
2    False
3     True
4     True
5    False
6     True
Name: number_of_employees, dtype: bool

In [39]:
data2 = data2.drop('_id', axis=1)

In [40]:
data2.head()

,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position,cities
0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ...",New York
1,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,...",Redwood City
2,Zango,games_video,225,1999,,$40M,"[{'description': None, 'address1': '3600 136th...",-122.155668,47.579208,"{'type': 'Point', 'coordinates': [-122.155668,...",Bellevue
3,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,...",Los Angeles
4,Crunchyroll,games_video,50,2008,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2...",-122.393229,37.781265,"{'type': 'Point', 'coordinates': [-122.393229,...",San Francisco


In [41]:
data2.to_json(r'./officesCity.json', orient="records")

In [42]:
san = san.drop('_id', axis=1)

In [43]:
san.head()

,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,Crunchyroll,games_video,50,2008,Digital Entertainment and Media Platform,$4.8M,"[{'description': '', 'address1': '', 'address2...",-122.393229,37.781265,"{'type': 'Point', 'coordinates': [-122.393229,..."
1,iWin,games_video,103,2001,,$0,"[{'description': '', 'address1': '45 Fremont S...",-122.391061,37.781689,"{'type': 'Point', 'coordinates': [-122.391061,..."
2,Curse,games_video,58,2006,Online games media company,$12M,"[{'description': 'San Francisco', 'address1': ...",-122.399972,37.787092,"{'type': 'Point', 'coordinates': [-122.399972,..."
3,hi5,games_video,100,2003,,$52M,"[{'description': '', 'address1': '55 Second St...",-122.400558,37.788668,"{'type': 'Point', 'coordinates': [-122.400558,..."
4,ngmoco,games_video,150,2008,Games for the iPhone,$44.8M,"[{'description': 'Secret Factory', 'address1':...",-122.401581,37.770510,"{'type': 'Point', 'coordinates': [-122.4015807..."


In [ ]:
san.to_csv('oficinas.csv')

In [ ]:
san.to_json(r'./officesFinal.json', orient="records")